In [1]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_from_disk

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [2]:
institutions_dataset = load_from_disk('../datasets/intermediate/institutions_dataset')

In [3]:
institutions_dataset

DatasetDict({
    test: Dataset({
        features: ['text', 'corrupt_institutions', 'not_corrupt_institutions'],
        num_rows: 494
    })
    train: Dataset({
        features: ['text', 'corrupt_institutions', 'not_corrupt_institutions'],
        num_rows: 7191
    })
    validation: Dataset({
        features: ['text', 'corrupt_institutions', 'not_corrupt_institutions'],
        num_rows: 919
    })
})

In [4]:
institutions_dataset['test'][0]

{'text': 'Belföld\nAz MSZP-vel fizettetné vissza Simon pénzét a Fidesz\nA Fidesz szerint, ha Simon Gábor, a szocialista párt volt elnökhelyettese nem fizeti vissza az összeférhetetlensége idején felvett 39 millió forintos parlamenti jövedelmét, akkor ezt az MSZP-nek kell megtennie helyette.\nGulyás Gergely, a kormánypárt országgyűlési képviselője pénteki budapesti sajtótájékoztatóján emlékeztetett, hogy Simon Gábor az elmúlt években egyik vagyonnyilatkozatában sem tüntette fel a titkos bécsi számlán 2008 óta tartott több mint 240 millió forintos vagyont.\nHa emiatt az Országgyűlés megállapította volna az összeférhetetlenségét, vissza kellett volna fizetnie az ez idő alatt felvett parlamenti jövedelmét, 39 millió forintot - magyarázta a Fidesz képviselője, aki szerint Simon Gábor éppen azért mondott le február 13-án (még mielőtt kimondták volna az összeférhetetlenséget) a mandátumáról, hogy ezt ne kelljen megtennie.\nA Fidesz most arra szólítja fel Simon Gábort és az MSZP-t, hogy előzzé

In [5]:
institutions_dataset = institutions_dataset.map(lambda row: {'all': row['corrupt_institutions'] + row['not_corrupt_institutions']})

In [6]:
#text = keyword_generation_task_dataset['test'][0]['text']

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from transformers import BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-v0.1"
peft_model_id = '../finetune/qlora-combined_people_institution-7b-v1.0.0/checkpoint-1131/adapter_model'
peft_model_id = '../finetune/qlora-out-entity-extraction.backup/'

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    ),
)
model = PeftModel.from_pretrained(model, peft_model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CUDA extension not installed.
CUDA extension not installed.


In [8]:
from tqdm import tqdm

device = 'cuda:0'
# features: ['text', 'corrupt_institutions', 'not_corrupt_institutions', 'all'],
institution_classification_prompt = '''[intézmény klasszifikáció]
{text}

###

összes: {all}
korrupcióban érintett:'''

tp = 0
fp = 0
tn = 0
fn = 0

for row in tqdm(institutions_dataset['test']):
    text = row['text']
    corrupt_institutions = row['corrupt_institutions']
    not_corrupt_institutions = row['not_corrupt_institutions']
    all_institutions = row['all']
    input_text = institution_classification_prompt.format(all=', '.join(all_institutions), text=text)
    with torch.cuda.amp.autocast():
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        if len(inputs['input_ids'][0]) > 2048:
            continue
        outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=200, penalty_alpha=0.6, top_k=3)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        sresult = result[result.rfind(':')+2:].strip()
        output_labels = sresult.strip().split(', ')
        for institution in corrupt_institutions:
            if institution in output_labels:
                tp += 1
            else:
                fn += 1
        for institution in not_corrupt_institutions:
            if institution in output_labels:
                fp += 1
            else:
                tn += 1
        print(tp, tn, fp, fn)

  0%|                                                                                                                                                     | 0/494 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
2023-12-02 09:54:01.240739: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 09:54:01.240835: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 09:54:01.241707: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register 

1 0 0 0


  0%|▌                                                                                                                                          | 2/494 [00:15<1:01:39,  7.52s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


3 6 0 1


  1%|▊                                                                                                                                          | 3/494 [00:30<1:31:28, 11.18s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

388 787 74 154


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 491/494 [49:31<00:32, 10.85s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


389 790 74 154


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 494/494 [49:38<00:00,  6.03s/it]

390 793 74 154


In [9]:
print('precision', tp/(tp+fp))
print('recall', tp/(tp+fn))
print('accuracy', (tp+tn)/(tp+tn+fp+fn))

precision 0.8405172413793104
recall 0.7169117647058824
accuracy 0.8384124734231042


In [10]:
all_possible_labels = set()
for labels in truths:
    for label in labels:
        all_possible_labels.add(label)
for labels in preds:
    for label in labels:
        all_possible_labels.add(label)
label_to_id = {key: value for value, key in enumerate(all_possible_labels)}
label_to_id

NameError: name 'truths' is not defined

In [ ]:
i_preds = [[1 if value in preds_a else 0 for value in label_to_id] for preds_a in preds]

In [ ]:
i_preds

In [ ]:
i_truths = [[1 if value in truths_a else 0 for value in label_to_id] for truths_a in truths]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(i_truths, i_preds, target_names=label_to_id))

In [ ]:
precision, recall, f1, _ = precision_recall_fscore_support(i_truths, i_preds)
acc = accuracy_score(labels, preds)
{
    'accuracy': acc,
    'precision': precision,
    'recall': recall,
    'f1': f1
}

In [ ]:
import numpy as np
y_true = np.array([[0, 1, 1, 1],[0,0,1,0],[1,1,0,0]])
y_scores = np.array([[0.2, 0.6, 0.1, 0.8],[0.4,0.9,0.8,0.6],[0.8,0.4,0.5,0.7]])
threshold = 0.5
y_pred=[]
for sample in  y_scores:
  y_pred.append([1 if i>=0.5 else 0 for i in sample ] )
y_pred = np.array(y_pred)
y_pred


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(i_truths, i_preds)


In [ ]:
print("Actual \n", y_true)
print("\nPredicted \n",y_pred)


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(i_truths, i_preds)
